In [48]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [50]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow.parquet as pq

In [51]:
df = pd.read_parquet("/content/dataset_ML.parquet")

In [52]:
df.head()

,id,title,userId,score
1669223,as3480,30 days of yoga for beginners | calm mind & bo...,109303,4.5
650417,as1357,pay the ghost,32160,3.5
4276094,as8923,koch brothers exposed,253054,3.0
2822661,as5887,fanboys,7372,3.0
1631434,as3402,a one-hour world tour,75663,3.0


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399997 entries, 1669223 to 9827083
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      399997 non-null  object 
 1   title   399997 non-null  object 
 2   userId  399997 non-null  int64  
 3   score   399997 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.3+ MB


## CREAMOS Y ENTRENAMOS EL MODELO

In [54]:
#Creamos una escala de la puntuacion, va desde el 1 al 5. 
reader = Reader(rating_scale=(1, 5))

Creo un objeto Dataset xon la librería "Surprise" utilizando la función load_from_df(), convirtiendo mi DataFrame de pandas a un objeto que pueda trabajar.

Selecciono solo 'userId', 'title' y 'score' df, posterior con la función load_from_df(). 

El orden de las columnas debe ser [usuario, ítem, puntuación], esta se encarga de unir nuestra variable que contenia el score, junto con las demas variables, y así convertirlo en un modelo entrenable por la librería.

In [55]:
data = Dataset.load_from_df(df[['userId', 'title', 'score']], reader)

Una vez, el data set armado, ya lo podemos utilizar para comenzar con el modelo.

Primero dividimos los parametros para 'train' y 'test'

In [56]:
trainset, testset = train_test_split(data, test_size=0.2)

### Instancio el modelo.

In [57]:
model = SVD()

### Entreno el modelo.

In [58]:
model.fit(trainset)

### Testeo el modelo.

In [59]:
predictions = model.test(testset)

### **EVALUACION**

Calculamos el error cuadrático medio (RMSE); el error absoluto medio (MAE) utilizando accuracy.rmse() y accuracy.mae() de Surprise. Los 3 cálculos que vimos en clases.

In [60]:
from surprise import accuracy #traigo las funciones que necesito

accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 1.0296
MAE:  0.8124


0.8124216389824113

Optimizamos de hiperparámetros.

In [61]:
from surprise.model_selection import cross_validate
import numpy as np

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128,256]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0242  1.0266  1.0257  1.0255  0.0010  
Fit time          3.54    3.20    3.70    3.48    0.21    
Test time         2.17    1.81    1.37    1.79    0.33    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0240  1.0253  1.0268  1.0254  0.0011  
Fit time          3.13    3.72    3.23    3.36    0.26    
Test time         1.10    1.63    1.12    1.28    0.24    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0250  1.0223  1.0288  1.0254  0.0027  
Fit time          3.48    3.20    3.67    3.45    0.19    
Test time         2.63    1.98    2.51    2.37    0.29    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0263  1.0

## **FUNCIÓN DE PREDICCIÓN**

### Todo unificado sería lo siguiente:

In [62]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
import pyarrow.parquet as pq
import pandas as pd
import numpy as np

# cargar el dataset de películas desde un archivo Parquet
df = pd.read_parquet("/content/dataset_ML.parquet")

# crear un objeto Reader y especificar la escala de calificación
reader = Reader(rating_scale=(1, 5))

# crear un objeto Dataset desde el dataframe
data = Dataset.load_from_df(df[['userId', 'title', 'score']], reader)

# dividir el dataset en train y test
trainset, testset = train_test_split(data, test_size=0.2)

# crear un modelo SVD
model = SVD()

# entrenar el modelo en el dataset de entrenamiento
model.fit(trainset)

# hacer predicciones sobre el dataset de test
predictions = model.test(testset)

# evaluar el modelo usando RMSE y MAE
accuracy.rmse(predictions)
accuracy.mae(predictions)

# optimizar hiperparámetros
rmse_test_means = []
n_factors_list = [1, 2, 4, 8, 16, 32, 64, 128, 256]

for n_factors in n_factors_list:
    print("n_factors: ", n_factors)
    model = SVD(n_factors=n_factors)
    cv = cross_validate(model, data, measures=['RMSE'], cv=3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

# mostrar los resultados de la optimización
for i in range(len(n_factors_list)):
    print("n_factors:", n_factors_list[i], ", RMSE:", rmse_test_means[i])


RMSE: 1.0248
MAE:  0.8083
n_factors:  1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0250  1.0252  1.0260  1.0254  0.0004  
Fit time          3.68    3.16    4.64    3.83    0.62    
Test time         1.26    1.10    1.48    1.28    0.16    
n_factors:  2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0264  1.0266  1.0243  1.0258  0.0010  
Fit time          3.62    4.80    4.61    4.34    0.52    
Test time         2.71    1.46    1.38    1.85    0.61    
n_factors:  4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0256  1.0250  1.0263  1.0256  0.0005  
Fit time          3.56    4.07    3.34    3.66    0.31    
Test time         1.17    1.03    1.33    1.18    0.12    
n_factors:  8
Evaluating RMSE of algorithm SVD on 3 split(s).

              

In [121]:
def recomendacion(id_usuario, clave):
    # Cargar el conjunto de datos completo
    data = Dataset.load_from_df(df[['userId', 'title', 'score']], reader)

    # Crear un modelo SVD con los mejores hiperparámetros
    model = SVD(n_factors=32)

    # Entrenar el modelo con todo el conjunto de datos
    trainset = data.build_full_trainset()
    model.fit(trainset)

    # Obtener las películas vistas por el usuario
    peliculas_vistas_por_usuario = df[df['userId'] == id_usuario]['title'].values

    # Crear un conjunto de películas no vistas por el usuario
    peliculas_no_vistas_por_usuario = np.setdiff1d(df['title'].unique(), peliculas_vistas_por_usuario)

    # Crear una lista de tuplas con el título de la película, su predicción de calificación y un consejo
    recomendaciones = []
    for pelicula_titulo in peliculas_no_vistas_por_usuario:
        prediccion = model.predict(id_usuario, pelicula_titulo)
        if prediccion.est >= 3.5:
            recomendaciones.append(pelicula_titulo)

    # Presentar los resultados en un mensaje
    if len(recomendaciones) > 0:
        mensaje = "¡Genial! En base a tus gustos, te recomendamos las siguientes películas:\n\n"
        for i, pelicula in enumerate(recomendaciones[:5], start=1):
            mensaje += f"{i}. {pelicula}\n"
    else:
        mensaje = "Lo sentimos, no pudimos encontrar ninguna película que se ajuste a tus gustos."
    
    return mensaje

In [122]:
recomendacion(12342, 'as321123')

'¡Genial! En base a tus gustos, te recomendamos las siguientes películas:\n\n1. "the paramedic angel"\n2. #alive\n3. #annefrank - parallel stories\n4. #blackaf\n5. #cats_the_mewvie\n'

In [128]:
recomendacion(32544, 'as321')

'¡Genial! En base a tus gustos, te recomendamos las siguientes películas:\n\n1. "the paramedic angel"\n2. #alive\n3. #annefrank - parallel stories\n4. #blackaf\n5. #cats_the_mewvie\n'